In [3]:
from isegm.model.modeling.swin_transformer import *
from isegm.model.modeling.swin_unet import *
from isegm.model.is_swinformer_model import SwinformerModel
from isegm.model.modeling.transformer_helper.cross_entropy_loss import CrossEntropyLoss


In [8]:
# Parameters for Swin-base:
IMG_SIZE = 384
DROP_PATH_RATE = 0.2
EMBED_DIM = 128
DEPTHS = [2, 2, 18, 2]
NUM_HEADS = [4, 8, 16, 32]
WINDOW_SIZE = 12

swin_base_backbone_params = dict(
    pretrain_img_size=IMG_SIZE,
    embed_dim=EMBED_DIM,
    depths=DEPTHS,
    num_heads=NUM_HEADS,
    window_size=WINDOW_SIZE
)

swin_base_backbone = SwinTransformer(**swin_base_backbone_params)


In [9]:
from isegm.model.modeling.transformer_helper.cross_entropy_loss import CrossEntropyLoss
import os
weight_folder = '/playpen-raid/qinliu/projects/iSegFormer/weights/'
swin_base_weight = os.path.join(
    weight_folder, 'swin_base_patch4_window12_384_22k.pth')
swin_large_weight = os.path.join(
    weight_folder, 'swin_base_patch4_window12_384_22k.pth')

swin_base_backbone.init_weights(swin_base_weight)

in_t = torch.randn((8, 3, 320, 480))
coord_t = torch.randn((8, 3, 320, 480))
out_t = swin_base_backbone(in_t, coord_t)
print([out_t[i].shape for i in range(len(out_t))])

swin_base_head_params = dict(
    in_channels=[128, 256, 512, 1024],
    in_index=[0, 1, 2, 3],
    channels=256,
    dropout_ratio=0.1,
    num_classes=1,
    loss_decode=CrossEntropyLoss(),
    align_corners=False,
)

swin_base_head = SwinTransfomerSegHead(**swin_base_head_params)
out = swin_base_head(out_t)
print(out.shape)


2022-02-07 19:54:54,137 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



[torch.Size([8, 128, 80, 120]), torch.Size([8, 256, 40, 60]), torch.Size([8, 512, 20, 30]), torch.Size([8, 1024, 10, 15])]
torch.Size([8, 1, 80, 120])


In [3]:
import torch

B, H, W, Ch = 8, 384, 384, 3
in_t = torch.randn((B, Ch, H, W))

backbone_params = dict(
    pretrain_img_size=W,
    ape=False,
    patch_size=4,
    embed_dim=128,
    window_size=12
)

backbone = SwinTransformer(**backbone_params)
out_t = backbone(in_t)
print([out_t[i].shape for i in range(len(out_t))])


[torch.Size([8, 96, 96, 96]), torch.Size([8, 192, 48, 48]), torch.Size([8, 384, 24, 24]), torch.Size([8, 768, 12, 12])]


In [17]:
from isegm.model.modeling.transformer_helper.cross_entropy_loss import CrossEntropyLoss
decode_head_params = dict(
    in_channels=[96, 192, 384, 768],
    in_index=[0, 1, 2, 3],
    channels=256,
    dropout_ratio=0.1,
    num_classes=1,
    loss_decode=CrossEntropyLoss(),
    align_corners=False,
)

head = SwinTransfomerSegHead(**decode_head_params)
out = head(out_t)
print(out.shape)


torch.Size([8, 1, 80, 120])


In [ ]:
B, W, L, Ch = 8, 224, 224, 3
in_t = torch.randn((B, Ch, W, L))
swin_unet = SwinTransformerSys(img_size=224, num_classes=2)
out_t = swin_unet(in_t)
print(out_t.shape)


SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:2
---final upsample expand_first---
torch.Size([8, 2, 224, 224])
